In [3]:
%pip install pypdf2

Note: you may need to restart the kernel to use updated packages.


### Carregando e extraindo dados

In [25]:
import re
import json
import os
import pandas as pd
import random

# ==============================================================================
# --- Configurações ---
# ==============================================================================
print("--- Configurações ---")
caminho_arquivo_bruto = 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/arquivo_bruto_extraido.txt' # Use o seu caminho
caminho_arquivo_pre_filtrado_opcional = 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/arquivo_pre_filtrado_debug.txt' # Use o seu caminho
caminho_arquivo_json = 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/dicionario_estruturado_final_v8.json' # <<<< Novo nome v8
caminho_arquivo_erros = 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/entradas_com_erro_v8.txt'         # <<<< Novo nome v8

output_dir = os.path.dirname(caminho_arquivo_json)
if output_dir and not os.path.exists(output_dir):
    print(f"Criando diretório de saída: {output_dir}")
    os.makedirs(output_dir)

# ==============================================================================
# --- PASSO 1, 2, 3 (Carregar, Filtrar, Dividir - Mantido como antes) ---
# ==============================================================================
print("\n--- PASSO 1: Carregando Texto Bruto ---")
texto_completo_bruto = None
encodings_to_try = ['utf-8', 'latin-1', 'cp1252']
for enc in encodings_to_try:
    try:
        with open(caminho_arquivo_bruto, 'r', encoding=enc) as f:
            texto_completo_bruto = f.read()
        print(f"Texto bruto carregado com sucesso de '{caminho_arquivo_bruto}' usando encoding '{enc}'.")
        print(f"Tamanho: {len(texto_completo_bruto)} caracteres.")
        break
    except FileNotFoundError: print(f"Erro Crítico: Arquivo bruto não encontrado em '{caminho_arquivo_bruto}'"); texto_completo_bruto = None; break
    except Exception as e: print(f"Falha ao ler com encoding '{enc}': {e}"); continue
if texto_completo_bruto is None: print("Erro Crítico: Não foi possível carregar o arquivo bruto."); exit()

print("\n--- PASSO 2: Iniciando Pré-filtragem em Memória ---")
texto_filtrado_memoria = ""; linhas_removidas_count = 0
if texto_completo_bruto:
    linhas_brutas = texto_completo_bruto.splitlines(); linhas_filtradas = []; num_linhas_brutas = len(linhas_brutas)
    for i, linha in enumerate(linhas_brutas):
        linha_strip = linha.strip()
        if not linha_strip: linhas_removidas_count += 1; continue
        if re.fullmatch(r'\d+', linha_strip): linhas_removidas_count += 1; continue
        if re.fullmatch(r'[\*k \-]+', linha_strip) and len(linha_strip) > 3: linhas_removidas_count += 1; continue
        if i > 0 and i < num_linhas_brutas - 1:
             if re.fullmatch(r'[A-ZÁÉÍÓÚÇÑŸ\-]+\s*(?:/\s*[A-ZÁÉÍÓÚÇÑŸ\-]+\??|-)\s*', linha_strip, re.IGNORECASE):
                 linha_anterior_vazia = not linhas_brutas[i-1].strip(); linha_posterior_vazia = not linhas_brutas[i+1].strip()
                 if linha_anterior_vazia or linha_posterior_vazia or len(linhas_brutas[i-1]) < 15 or len(linhas_brutas[i+1]) < 15:
                    linhas_removidas_count += 1; continue
        if "Biblioteca Digital Curt Nimuendajú" in linha or "http://www.etnolinguistica.org" in linha:
            linhas_removidas_count += 1; continue
        linhas_filtradas.append(linha)
    texto_filtrado_memoria = "\n".join(linhas_filtradas)
    print(f"Pré-filtragem concluída. {linhas_removidas_count} linhas removidas."); print(f"Tamanho do texto filtrado: {len(texto_filtrado_memoria)} caracteres.")
    if caminho_arquivo_pre_filtrado_opcional:
        try:
            with open(caminho_arquivo_pre_filtrado_opcional, 'w', encoding='utf-8') as f_out: f_out.write(texto_filtrado_memoria)
            print(f"Texto pré-filtrado (para debug) salvo em '{caminho_arquivo_pre_filtrado_opcional}'")
        except Exception as e: print(f"Erro ao salvar arquivo pré-filtrado opcional: {e}")
else: print("Texto bruto não carregado.")

print("\n--- PASSO 3: Dividindo Texto em Entradas ---")
entradas_brutas = []
if texto_filtrado_memoria:
    regex_inicio_entrada = re.compile(r"^(?:\s{0,2}(?:—\s)?(?:[A-ZÁÉÍÓÚÇÑŸ]|[ÁÉÍÓÚ]\b))|^(?:\s{0,2}[A-Z]\s*)$", re.MULTILINE)
    indices_inicio = [m.start() for m in regex_inicio_entrada.finditer(texto_filtrado_memoria)]
    if indices_inicio:
        print(f"Encontrados {len(indices_inicio)} potenciais inícios de entrada no texto filtrado.")
        for i in range(len(indices_inicio)):
            start = indices_inicio[i]; end = indices_inicio[i+1] if (i+1) < len(indices_inicio) else len(texto_filtrado_memoria)
            bloco = texto_filtrado_memoria[start:end].strip()
            if bloco: entradas_brutas.append(bloco)
        print(f"Dividido em {len(entradas_brutas)} entradas brutas.")
        if entradas_brutas:
            print("\n--- Exemplo das primeiras 5 entradas brutas (Após Divisão v8) ---")
            for i, entrada in enumerate(entradas_brutas[:5]): print(f"--- Entrada Bruta {i+1} ---"); print(entrada); print("-" * 20)
    else: print("Nenhum início de entrada encontrado com o padrão atual no texto filtrado.")
else: print("Texto filtrado em memória está vazio.")

# ==============================================================================
# --- PASSO 4: Função de Limpeza Robusta (v8 - EXPANDIR!) ---
# ==============================================================================
print("\n--- PASSO 4: Definindo Função de Limpeza v8 ---")
# Lista de classes (precisa ser definida globalmente)
CLASSES_GRAMATICAIS_LIST = sorted([
    'Prefixo', 'Sufixo verbal', 'Sufixo', 'Afixo', 'Substantivo irregular', 'Substantivo', 'Gerúndio',
    'Verbo transitivo irregular', 'Verbo intransitivo irregular', 'Verbo transitivo relativo', 'Verbo transitivo',
    'Verbo intransitivo', 'Verbo predicativo', 'Verbo factivo', 'Verbo pronominal', 'Verbo reflexivo',
    'Verbo iterativo', 'Verbo defectivo', 'Verbo', 'Verbo durativo', 'Verbo freqüentativo', 'Verbo birrelativo',
    'Vocativo', 'Demonstrativo', 'Partículas agrupadas', 'Partícula', 'Advérbio', 'Locução adverbial',
    'Adjetivo', 'Pronome interrogativo', 'Pronome pessoal agente', 'Pronome pessoal', 'Pronome relativo', 'Pronome indefinido',
    'Pronome possessivo', 'Pronome objetivo', 'Pronome',
    'Numeral cardinal', 'Numeral ordinal', 'Numeral distributivo', 'Numeral',
    'Interjeigáo', 'Interjeição', 'Conjungáo', 'Conjunção', 'Preposiqáo', 'Preposição', 'Locuqáo prepositiva',
    'Locuqáo adverbial', 'Locuqáo substantiva', 'Particípio', 'Ictiologia',
    'Ornitologia', 'Zoologia', 'Botánica', 'Entomologia', 'Conquiliologia',
    'Crustaceologia', 'Astronomia', 'Mineralogia', 'Mitologia', 'Malacologia',
    'Fitologia', 'Anatomia'
], key=len, reverse=True)

def limpar_texto_robusto(texto, classes_para_padronizar):
    if not texto: return ""
    texto_processado = " ".join(line.strip() for line in texto.splitlines() if line.strip())

    # !!!!!!!!!! EXPANDA MASSIVAMENTE ESTE DICIONÁRIO !!!!!!!!!!
    correcoes = {
         # Erros de OCR + Normalização de Classes (Exemplos)
         'Prefi. xo': 'Prefixo', 'fiubstantivo': 'Substantivo', 'Substarítivo': 'Substantivo', 'S)ibsta\'ntivo': 'Substantivo',
         'Substantlvo': 'Substantivo', 'Sübstantivo': 'Substantivo', 'Substantivo irregular:':'Substantivo irregular:', 'fíubstantivo':'Substantivo', 'Substantivo:':'Substantivo:',
         'Adj^fcivo': 'Adjetivo', 'A_d jetivo':'Adjetivo', 'Mjetivao': 'Adjetivo','Adjetivo:':'Adjetivo:',
         'Partlculas': 'Partículas', 'Patticulas': 'Partículas', 'Partícúla': 'Partícula', 'Particlpio':'Particípio', 'Particípio':'Particípio:', 'Particulas':'Partículas','Particulas agrupadas':'Partículas agrupadas:',
         'Advérbiorse': 'Advérbio: se','Adverbio:': 'Advérbio:', 'Adverbio': 'Advérbio', 'Advérbio:':'Advérbio:',
         'Verbotransitivo': 'Verbo transitivo', 'Verbo intransitivo:o': 'Verbo intransitivo:', 'Verbo intransitivo :':'Verbo intransitivo:',
         'Verbo paredioativo': 'Verbo predicativo', 'Verbo transitivo:':'Verbo transitivo:', 'Verbo intransitivo:':'Verbo intransitivo:',
         'Verbo reflexivo:':'Verbo reflexivo:', 'Verbo predicativo:':'Verbo predicativo:', 'Verbo factivo:':'Verbo factivo:',
         'Verbo pronominal:':'Verbo pronominal:', 'Verbo iterativo:':'Verbo iterativo:', 'Verbo defectivo:':'Verbo defectivo:', 'Verbo:':'Verbo:',
         'Locucao':'Locução', 'Locugao':'Locução','Locuçáo prepositiva:':'Locução prepositiva:', 'Locucáo adverbial:':'Locução adverbial:',
         'Locucáo substantiva:':'Locução substantiva:', 'Locução substantiva:':'Locução substantiva:', 'Locução prepositiva:':'Locução prepositiva:',
         'Locução adverbial:':'Locução adverbial:',
         'Pronome interrogativo:': 'Pronome interrogativo:', 'Pronome pessoal:': 'Pronome pessoal:', 'Pronome pessoal agente:':'Pronome pessoal agente:',
         'Pronome relativo:': 'Pronome relativo:', 'Pronome indefinido:': 'Pronome indefinido:',
         'Pronome possessivo:': 'Pronome possessivo:', 'Pronome objetivo:': 'Pronome objetivo:',
         'Pronome:': 'Pronome:',
         'Numeral:': 'Numeral:', 'Interjeigáo:': 'Interjeição:', 'Conjungáo:': 'Conjunção:', 'Conjungao:':'Conjunção:',
         'Preposiqáo:': 'Preposição:', 'Preposicáo:': 'Preposição:', 'Ictiologia:':'Ictiologia:',
         'Ornitologia:':'Ornitologia:', 'Zoologia:':'Zoologia:', 'Botánica:':'Botânica:',
         'Entomologia:':'Entomologia:', 'Conquiliologia:':'Conquiliologia:',
         'Crustaceologia:':'Crustaceologia:', 'Astronomia:':'Astronomia:', 'Mineralogia:':'Mineralogia:',
         'Mitologia:':'Mitologia:', 'Malacologia:':'Malacologia:', 'Fitologia:':'Fitologia:', 'Anatomia:':'Anatomia:',
         'Vocativo:':'Vocativo:', 'Gerúndio:':'Gerúndio:', 'Demonstrativo:':'Demonstrativo:',
         'Afixo:':'Afixo:', 'Sufixo:':'Sufixo:', 'Sufixo verbal:':'Sufixo verbal:', 'Numeral cardinal:':'Numeral cardinal:',
         'Numeral ordinal:':'Numeral ordinal:', 'Numeral distributivo:':'Numeral distributivo:',
         # --- Erros de OCR e Acentuação ---
         'observagoes': 'observações', 'índío': 'índio', 'nao': 'não', 'Nao': 'Não', 'náo': 'não', 'Náo':'Não', 'hã':'há', 'homen':'homem',
         'fungáo': 'função', 'conjugagao': 'conjugação', 'Cabega': 'Cabeça', 'cabeqa': 'cabeça', 'cabe<ja': 'cabeça', 'grao':'grão','señhorl':'senhor!',
         'composigao': 'composição', 'posipoes': 'posições', 'só ha': 'só há', ' ha ': ' há ', 'estã':'está','glande':'glande', 'ami- go':'amigo',
         'intradutlvel': 'intradutível', 'próxi mo': 'próximo', 'aqáo': 'ação', 'signíficado': 'significado','formagao':'formação',
         'á': 'ã', 'Á': 'Ã', '§': 'ç', '£': 'S', # Suposições
         ' 9 ': ' ç ', '9 ': 'ç ', ' 9': ' ç', '(t, 9 )':'(t, ç)', # Correção 9
         'ü': 'u', 'Ü': 'U', # Normalizar trema
         ' l ': ' i ', # l isolado
         'permiss^L': 'permissivo', '^fruto':'1 fruto', ' z ': ' 2 ', 'zGlande':'2 Glande','^contradizer':'1 contradizer', # Numeração
         'si nal': 'sinal', 'planta (desenho)': 'planta (desenho)', 'prepoderante': 'preponderante', 'oontrário': 'contrário', 'riiinca': 'nunca',
         'afcsolutamente': 'absolutamente', 'a/grupadas': 'agrupadas', 'sicj': 'sic', 'multidao': 'multidão',
         'duplicagao': 'duplicação', 'duplicaqao': 'duplicação', 'procedéncia': 'procedência', 'género': 'gênero',
         'famllia': 'família', 'séxo': 'sexo', 'mé': 'mãe', 'fago': 'fago', 'recordagao': 'recordação', 'padre': 'Padre',
         'traduqáo': 'tradução', 'conjungáp': 'conjunção', 'Erigado': 'Eriçado', 'Bispo': 'Bispo', 'tupi) . É':'tupi). É',
         'diminutl:vo': 'diminutivo', 'répleto': 'repleto', 'pélos': 'pelos', 'respiraqao': 'respiração', 'memória': 'memória',
         'fiaqao': 'fiação', 'recepgáo': 'recepção', 'AgÉ-AgEMA': 'AGÉ-AGEMA', 'las.': '1as.',
         'preposi^ gao': 'preposição', 'pessbal': 'pessoal', 'prepos_i gáo': 'preposição', 'Adáo': 'Adão',
         'abundáncia': 'abundância', 'Preposdgáo': 'Preposição', 'desen^Dlvem': 'desenvolvem',
         'cansago': 'cansaço', 'conjugáo': 'conjunção', 'oragáo': 'oração', 'prSprio': 'próprio', 'oragoes': 'orações',
         'ünica': 'única', 'preposiqoes': 'preposições', 'preposiqáo': 'preposição', 'entá o': 'então',
         'naqu_i': 'naquilo', 'interroga tiva': 'interrogativa', 'encru2ilhada': 'encruzilhada', 'fungoes': 'funções',
         'agao': 'ação', 'posiqáo': 'posição', 'explrlito': 'espírito', 'relaqao': 'relação', 'engros- sar': 'engrossar',
         'solugáo': 'solução', 'ÍABA': 'ABA', 'f ': '† ', '+ ': '‡ ',
         '(XE).': '(XE)', '(T-).': '(T-)', '(Q-).': '(Q-)', '(C-).': '(C-)', '(IO-).': '(IO-)', '(NH-).': '(NH-)', '(I-).': '(I-)',
         'lingua': 'língua', 'segundo': 'segundo', 'homém': 'homem', 'fácil': 'fácil', 'ingreme': 'íngreme', 'máo': 'mão',
         'güloso': 'guloso', 'trapaceiro': 'trapaceiro', 'ladráo': 'ladrão', 'descorogoa do': 'descoroçoado',
         'córtez': 'cortês', 'distinfto': 'distinto', 'antropo fago': 'antropófago', 'povo': 'povo',
         'entranhas': 'entranhas', 'pélo': 'pelo', 'sobrinatural': 'sobrenatural', 'tarróz': 'arroz',
         'pjretos': 'pretos', 'Particula': 'Partícula', 'aemais': 'demais', 'slgnifica': 'significa',
         'sinónimo': 'sinônimo', 'luslsmos': 'lusismos', 'frema': 'trema', 'condicicnal': 'condicional',
         'adjetivos': 'adjetivos', 'constitui-se': 'constitui-se', 'excrémenlo': 'excremento', 'portu- gués': 'português',
         'porem': 'porém', 'pas- sado': 'passado',
         # Normalização de acentos
         'á':'a', 'Á':'A', 'é':'e', 'É':'E', 'í':'i', 'Í':'I', 'ó':'o', 'Ó':'O', 'ú':'u', 'Ú':'U',
         'ã':'a', 'Ã':'A', 'õ':'o', 'Õ':'O', 'ê':'e', 'Ê':'E', 'ô':'o', 'Ô':'O', # Remover nasais e circunflexos? Avaliar!
         'ç':'ç', 'Ç':'Ç',
         # ... (Adicionar mais regras!)
    }

    # Aplica correções
    texto_processado = texto_processado.replace('(t, 9 )', '(t, ç)') # Correção específica
    for erro, correcao in correcoes.items():
        texto_processado = texto_processado.replace(erro, correcao) # Case-sensitive

    # Normalização final de espaços e pontuação
    texto_processado = re.sub(r'\s+', ' ', texto_processado).strip()
    texto_processado = re.sub(r'([.,;:?!])([a-zA-Z0-9])', r'\1 \2', texto_processado)
    texto_processado = re.sub(r'\s([.,;:?!])', r'\1', texto_processado)
    texto_processado = re.sub(r'\s+:\s+', ': ', texto_processado)
    texto_processado = re.sub(r'([a-zA-Z]):([a-zA-Z])', r'\1: \2', texto_processado)

    # Padronizar classes conhecidas para terminar com ':'
    for cl in classes_para_padronizar:
        # Substitui Classe. ou Classe (seguido de espaço/fim) por Classe:
        pattern_end_dot = r'\b' + re.escape(cl) + r'\.(?=\s|$)'
        pattern_end_space = r'\b' + re.escape(cl) + r'(?=\s|$)(?!:)'
        texto_processado = re.sub(pattern_end_dot, cl + ':', texto_processado, flags=re.IGNORECASE)
        texto_processado = re.sub(pattern_end_space, cl + ':', texto_processado, flags=re.IGNORECASE)
    texto_processado = re.sub(r':\s+', ': ', texto_processado)

    return texto_processado.strip()

print("Função de limpeza definida.")


# --- PASSO 5: Processamento e Extração Refinados (v8) ---
dados_estruturados = []
entradas_com_erro = []

# Regex para encontrar classe (permite espaços/hífens INTERNOS) seguida por ':'
# O grupo 1 captura o nome completo da classe
regex_classe_precisa_v8 = re.compile(r'(\b(?:' + '|'.join(re.escape(c).replace(r'\ ', r'\s+') for c in CLASSES_GRAMATICAIS_LIST) + r')(?:\s+[-\w]+)*)\s*:', re.IGNORECASE)
# Regex para limpar marcadores do início das definições (1., †, etc.)
regex_limpa_def_inicio_v8 = re.compile(r'^\s*(?:[1-9]+\.|†|‡|\^|z\b)\s*')

if 'entradas_brutas' in locals() and entradas_brutas:
    print(f"\n--- PASSO 5: Iniciando Processamento e Extração v8 de {len(entradas_brutas)} Entradas ---")
    for i, entrada_bruta in enumerate(entradas_brutas):
        if (i + 1) % 200 == 0: print(f"Processando entrada {i+1}/{len(entradas_brutas)}...")

        try:
            # Limpa a entrada inteira PRIMEIRO
            entrada_limpa = limpar_texto_robusto(entrada_bruta, CLASSES_GRAMATICAIS_LIST)
            if not entrada_limpa: continue

            verbete_tupi = ""
            marcadores = None
            classe_gramatical = None
            texto_para_definicoes = entrada_limpa
            primeira_linha_original = entrada_bruta.splitlines()[0].strip() if entrada_bruta else entrada_limpa

            # 1. Extrair Marcadores (...) primeiro
            match_marcador = re.search(r'(\s*[({\[].*?[)}\]])', texto_para_definicoes)
            if match_marcador:
                marcadores = match_marcador.group(1).strip()
                start, end = match_marcador.span()
                texto_para_definicoes = (texto_para_definicoes[:start] + texto_para_definicoes[end:]).strip()

            # 2. Encontrar a PRIMEIRA Classe Gramatical + ':' no texto restante
            match_classe_obj = regex_classe_precisa_v8.search(texto_para_definicoes)
            pos_classe_inicio = match_classe_obj.start() if match_classe_obj else -1

            if pos_classe_inicio != -1:
                classe_achada = match_classe_obj.group(1).strip()
                classe_gramatical = classe_achada.capitalize()
                verbete_tupi = texto_para_definicoes[:pos_classe_inicio].strip()
                # **CORREÇÃO:** Texto para definições é tudo DEPOIS do match completo (incluindo o ':')
                texto_para_definicoes = texto_para_definicoes[match_classe_obj.end():].strip()
            else:
                # Nenhuma classe encontrada. Tenta separar por marcador de definição ou heurística.
                match_def_marker = re.search(r'\s(?:[1-9]+\.|†|‡|\^|z\b|EX:| = )', texto_para_definicoes)
                if match_def_marker:
                    verbete_tupi = texto_para_definicoes[:match_def_marker.start()].strip()
                    texto_para_definicoes = texto_para_definicoes[match_def_marker.start():].strip()
                else:
                    # Assume verbete é tudo.
                    verbete_tupi = texto_para_definicoes
                    texto_para_definicoes = ""


            # 3. Limpeza final do verbete (MAIS CUIDADOSA)
            # Remove APENAS : ou ? do final da string, preserva . e —
            verbete_tupi = re.sub(r'[:?]\s*$', '', verbete_tupi).strip()
            # Remove traço inicial APENAS se seguido por espaço
            if verbete_tupi.startswith('— '): verbete_tupi = verbete_tupi[2:].strip()
            # Remove traço final APENAS se precedido por espaço E não houver hífen interno
            if verbete_tupi.endswith(' —') and '-' not in verbete_tupi[:-2]: verbete_tupi = verbete_tupi[:-2].strip()


            # 4. Extrair Definições, Exemplos, Referências
            definicoes = []
            exemplos = []
            referencias = []

            # Regex para marcadores de início de item (número, símbolo, EX, =)
            regex_split_corpo = re.compile(r'(?=\s*(?:[1-9]+\.|†|‡|\^|z\b|EX:| = ))')
            partes_corpo = regex_split_corpo.split(texto_para_definicoes)

            for parte in partes_corpo:
                parte = parte.strip()
                if not parte: continue

                match_ex = re.match(r'^\s*EX:\s*(.*)', parte, re.IGNORECASE)
                match_ref = re.match(r'^\s*=\s*(.*)', parte)
                # Tenta capturar o texto APÓS o marcador numérico/símbolo
                match_num = re.match(r'^\s*(?:[1-9]+\.|†|‡|\^|z\b)\s*(.*)', parte)

                if match_ex:
                    texto_item = match_ex.group(1).strip();
                    if texto_item: exemplos.append(texto_item)
                elif match_ref:
                    texto_item = match_ref.group(1).strip();
                    if texto_item: referencias.append(texto_item)
                elif match_num:
                    texto_item = match_num.group(1).strip(); # Pega o grupo 1 (texto após marcador)
                    if texto_item: definicoes.append(texto_item)
                else:
                     # Se não começou com marcador conhecido, é definição
                     if parte: definicoes.append(parte)


            # 5. Armazenar dados estruturados
            if verbete_tupi or definicoes or exemplos or referencias:
                 if not re.fullmatch(r'[A-Z]\s*$', primeira_linha_original): # Filtra A, B, C..
                    dados_estruturados.append({
                        'id': len(dados_estruturados),
                        'verbete_original_linha': primeira_linha_original,
                        'verbete_tupi': verbete_tupi,
                        'marcadores': marcadores,
                        'classe_gramatical': classe_gramatical,
                        'definicoes': definicoes,
                        'exemplos': exemplos,
                        'referencias': referencias,
                    })

        except Exception as e:
            print(f"\n!!!!!! Erro Crítico ao processar entrada bruta {i} !!!!!!")
            print(f"Erro: {e}")
            texto_erro = entrada_bruta[:1000] + "..." if len(entrada_bruta) > 1000 else entrada_bruta
            entradas_com_erro.append({"id_bruta": i, "erro": str(e), "texto": texto_erro})

    print(f"\nProcessadas {len(dados_estruturados)} entradas estruturadas antes da filtragem final (v8).")
    print(f"Encontrados {len(entradas_com_erro)} erros durante o processamento.")

    # --- PASSO 6: Filtragem Final ---
    print("\n--- PASSO 6: Filtrando Entradas Vazias/Inválidas ---")
    dados_filtrados_final = [
        entry for entry in dados_estruturados
        if entry.get('verbete_tupi') and \
           (entry.get('classe_gramatical') or entry.get('definicoes') or entry.get('exemplos') or entry.get('referencias'))
    ]
    num_removidos_filtragem = len(dados_estruturados) - len(dados_filtrados_final)
    print(f"Removidas {num_removidos_filtragem} entradas durante a filtragem final.")
    dados_estruturados = dados_filtrados_final # Usa a lista filtrada

    # --- PASSO 7: Salvar Resultados ---
    print("\n--- PASSO 7: Salvando Resultados ---")
    # ----- Salvar os dados estruturados em JSON -----
    try:
        if output_dir and not os.path.exists(output_dir): print(f"Criando diretório de saída para JSON: {output_dir}"); os.makedirs(output_dir)
        with open(caminho_arquivo_json, 'w', encoding='utf-8') as f_json: json.dump(dados_estruturados, f_json, ensure_ascii=False, indent=2)
        print(f"\nDados estruturados finais ({len(dados_estruturados)} entradas) salvos em '{caminho_arquivo_json}'")
        if dados_estruturados:
            print("\n--- Exemplo das Primeiras 10 Entradas Estruturadas Finais (JSON v8) ---")
            for k in range(min(10, len(dados_estruturados))):
                 print(f"--- Entrada ID {dados_estruturados[k]['id']} ---"); print(json.dumps(dados_estruturados[k], ensure_ascii=False, indent=2)); print("-" * 20)
        else: print("\nNenhuma entrada estruturada válida para mostrar após filtragem.")
    except Exception as e: print(f"Erro ao salvar o arquivo JSON final: {e}")

    # ----- Salvar entradas com erro (se houver) -----
    if entradas_com_erro:
        try:
            output_dir_erro = os.path.dirname(caminho_arquivo_erros);
            if output_dir_erro and not os.path.exists(output_dir_erro): print(f"Criando diretório de saída para Erros: {output_dir_erro}"); os.makedirs(output_dir_erro)
            with open(caminho_arquivo_erros, 'w', encoding='utf-8') as f_err: json.dump(entradas_com_erro, f_err, ensure_ascii=False, indent=2)
            print(f"Informações de {len(entradas_com_erro)} erro(s) salvas em '{caminho_arquivo_erros}'")
        except Exception as e: print(f"Erro ao salvar arquivo de erros: {e}")

elif not 'entradas_brutas' in locals() or not entradas_brutas:
     print("Não há entradas brutas para processar. Verifique os Passos 1, 2 e 3.")

print("\n--- Processamento Concluído ---")

# --- PASSO 8: Validação Rápida com Pandas (Opcional) ---
# (Código de validação mantido como antes)
print("\n--- PASSO 8: Iniciando Validação Rápida com Pandas (Opcional) ---")
if os.path.exists(caminho_arquivo_json):
    try:
        df_final = pd.read_json(caminho_arquivo_json)
        print(f"JSON final carregado no Pandas com {len(df_final)} entradas.")
        pd.set_option('display.max_rows', 100); pd.set_option('display.max_colwidth', 200)
        print("\n--- Amostra Aleatória Final (15 entradas) ---")
        num_amostras = min(15, len(df_final))
        if num_amostras > 0: print(df_final.sample(n=num_amostras))
        else: print("DataFrame final vazio.")
        print("\n--- Contagem de Nulos por Coluna ---"); print(df_final.isnull().sum())
        if 'classe_gramatical' in df_final.columns: print("\n--- Exemplo de Entradas SEM Classe Gramatical ---"); print(df_final[df_final['classe_gramatical'].isnull()].head())
        if 'definicoes' in df_final.columns: print("\n--- Exemplo de Entradas COM Definições Vazias ---"); print(df_final[df_final['definicoes'].apply(lambda x: not x)].head()) # Verifica lista vazia
    except Exception as e: print(f"Erro durante a validação com Pandas: {e}")
else: print(f"Arquivo JSON final '{caminho_arquivo_json}' não encontrado para validação.")

--- Configurações ---

--- PASSO 1: Carregando Texto Bruto ---
Texto bruto carregado com sucesso de 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/arquivo_bruto_extraido.txt' usando encoding 'utf-8'.
Tamanho: 666298 caracteres.

--- PASSO 2: Iniciando Pré-filtragem em Memória ---
Pré-filtragem concluída. 10697 linhas removidas.
Tamanho do texto filtrado: 655031 caracteres.
Texto pré-filtrado (para debug) salvo em 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/arquivo_pre_filtrado_debug.txt'

--- PASSO 3: Dividindo Texto em Entradas ---
Encontrados 7946 potenciais inícios de entrada no texto filtrado.
Dividido em 7946 entradas brutas.

--- Exemplo das primeiras 5 entradas brutas (Após Divisão v8) ---
--- Entrada Bruta 1 ---
A
--------------------
--- Entrada Bruta 2 ---
AAN-GATU
--------------------
--- Entrada Bruta 3 ---
A
--------------------
--- Entrada Bruta 4 ---
A — . Prefi.xo: atua como pronome pessoal agente, da primei 
ra pessoa de verbos da segunda c

### Normalizando

In [ ]:
import re
import json
import os
import pandas as pd
import random

# --- Configurações ---
caminho_arquivo_bruto = 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/arquivo_bruto_extraido.txt'
caminho_arquivo_pre_filtrado_opcional = 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/arquivo_pre_filtrado_debug.txt'
caminho_arquivo_json = 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/dicionario_estruturado_final_v6.json' # <<<< Novo nome v5
caminho_arquivo_erros = 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/entradas_com_erro_v6.txt'         # <<<< Novo nome v5
# --- PASSO 1, 2, 3 (Carregar, Filtrar, Dividir - Mantido como antes) ---
print("--- PASSO 1: Carregando Texto Bruto ---")
# ... (código de carregamento v3) ...
texto_completo_bruto = None
encodings_to_try = ['utf-8', 'latin-1', 'cp1252']
for enc in encodings_to_try:
    try:
        with open(caminho_arquivo_bruto, 'r', encoding=enc) as f:
            texto_completo_bruto = f.read()
        print(f"Texto bruto carregado com sucesso de '{caminho_arquivo_bruto}' usando encoding '{enc}'.")
        print(f"Tamanho: {len(texto_completo_bruto)} caracteres.")
        break
    except FileNotFoundError: print(f"Erro Crítico: Arquivo bruto não encontrado em '{caminho_arquivo_bruto}'"); texto_completo_bruto = None; break
    except Exception as e: print(f"Falha ao ler com encoding '{enc}': {e}"); continue
if texto_completo_bruto is None: print("Erro Crítico: Não foi possível carregar o arquivo bruto."); exit()

print("\n--- PASSO 2: Iniciando Pré-filtragem em Memória ---")
# ... (código de pré-filtragem v3) ...
texto_filtrado_memoria = ""
linhas_removidas_count = 0
if texto_completo_bruto:
    linhas_brutas = texto_completo_bruto.splitlines(); linhas_filtradas = []; num_linhas_brutas = len(linhas_brutas)
    for i, linha in enumerate(linhas_brutas):
        linha_strip = linha.strip()
        if not linha_strip: linhas_removidas_count += 1; continue
        if re.fullmatch(r'\d+', linha_strip): linhas_removidas_count += 1; continue
        if re.fullmatch(r'[\*k \-]+', linha_strip) and len(linha_strip) > 3: linhas_removidas_count += 1; continue
        if i > 0 and i < num_linhas_brutas - 1:
             if re.fullmatch(r'[A-ZÁÉÍÓÚÇÑŸ\-]+\s*(?:/\s*[A-ZÁÉÍÓÚÇÑŸ\-]+\??|-)\s*', linha_strip, re.IGNORECASE):
                 linha_anterior_vazia = not linhas_brutas[i-1].strip(); linha_posterior_vazia = not linhas_brutas[i+1].strip()
                 if linha_anterior_vazia or linha_posterior_vazia or len(linhas_brutas[i-1]) < 15 or len(linhas_brutas[i+1]) < 15:
                    linhas_removidas_count += 1; continue
        if "Biblioteca Digital Curt Nimuendajú" in linha or "http://www.etnolinguistica.org" in linha:
            linhas_removidas_count += 1; continue
        linhas_filtradas.append(linha)
    texto_filtrado_memoria = "\n".join(linhas_filtradas)
    print(f"Pré-filtragem concluída. {linhas_removidas_count} linhas removidas.")
    print(f"Tamanho do texto filtrado: {len(texto_filtrado_memoria)} caracteres.")
    if caminho_arquivo_pre_filtrado_opcional:
        try:
            with open(caminho_arquivo_pre_filtrado_opcional, 'w', encoding='utf-8') as f_out: f_out.write(texto_filtrado_memoria)
            print(f"Texto pré-filtrado (para debug) salvo em '{caminho_arquivo_pre_filtrado_opcional}'")
        except Exception as e: print(f"Erro ao salvar arquivo pré-filtrado opcional: {e}")
else: print("Texto bruto não carregado.")

print("\n--- PASSO 3: Dividindo Texto em Entradas ---")
# ... (código de divisão v3) ...
entradas_brutas = []
if texto_filtrado_memoria:
    regex_inicio_entrada = re.compile(r"^(?:\s{0,3}(?:—\s)?(?:[A-ZÁÉÍÓÚÇÑŸ]|[ÁÉÍÓÚ]\b))|^(?:\s{0,3}[A-Z]\s*)$", re.MULTILINE)
    indices_inicio = [m.start() for m in regex_inicio_entrada.finditer(texto_filtrado_memoria)]
    if indices_inicio:
        print(f"Encontrados {len(indices_inicio)} potenciais inícios de entrada no texto filtrado.")
        for i in range(len(indices_inicio)):
            start = indices_inicio[i]; end = indices_inicio[i+1] if (i+1) < len(indices_inicio) else len(texto_filtrado_memoria)
            bloco = texto_filtrado_memoria[start:end].strip()
            if bloco: entradas_brutas.append(bloco)
        print(f"Dividido em {len(entradas_brutas)} entradas brutas.")
        if entradas_brutas:
            print("\n--- Exemplo das primeiras 5 entradas brutas (Após Divisão v6) ---")
            for i, entrada in enumerate(entradas_brutas[:5]): print(f"--- Entrada Bruta {i+1} ---"); print(entrada); print("-" * 20)
    else: print("Nenhum início de entrada encontrado com o padrão atual no texto filtrado.")
else: print("Texto filtrado em memória está vazio.")


# --- PASSO 4: Função de Limpeza Robusta (EXPANDIR SEMPRE!) ---
print("\n--- PASSO 4: Definindo Função de Limpeza v6 ---")
# (Cole a função limpar_texto_robusto da resposta anterior aqui, E ADICIONE MAIS CORREÇÕES)
def limpar_texto_robusto(texto):
    if not texto: return ""
    # Junta linhas com espaço simples, mas preserva quebras para análise posterior se necessário
    texto_processado = " ".join(line.strip() for line in texto.splitlines() if line.strip())

    # Dicionário de Correções (EXPANDA MASSIVAMENTE!)
    correcoes = {
        # Erros comuns de OCR e Acentuação (exemplos)
         'Prefi. xo': 'Prefixo', 'fiubstantivo': 'Substantivo', 'Substarítivo': 'Substantivo', 'S)ibsta\'ntivo': 'Substantivo',
         'Substantlvo': 'Substantivo', 'Sübstantivo': 'Substantivo', 'Substantivo irregular:':'Substantivo irregular:', 'fíubstantivo':'Substantivo',
         'Adj^fcivo': 'Adjetivo', 'A_d jetivo':'Adjetivo', 'Mjetivao': 'Adjetivo','Adjetivo:':'Adjetivo:',
         'Partlculas': 'Partículas', 'Patticulas': 'Partículas', 'Partícúla': 'Partícula', 'Particlpio':'Particípio', 'Particípio':'Particípio:',
         'Advérbiorse': 'Advérbio: se','Adverbio:': 'Advérbio:', 'Adverbio': 'Advérbio', 'Advérbio:':'Advérbio:',
         'Verbotransitivo': 'Verbo transitivo', 'Verbo intransitivo:o': 'Verbo intransitivo:', 'Verbo intransitivo :':'Verbo intransitivo:',
         'Verbo paredioativo': 'Verbo predicativo', 'Verbo transitivo:':'Verbo transitivo:', 'Verbo intransitivo:':'Verbo intransitivo:',
         'Verbo reflexivo:':'Verbo reflexivo:', 'Verbo predicativo:':'Verbo predicativo:', 'Verbo factivo:':'Verbo factivo:',
         'Verbo pronominal:':'Verbo pronominal:', 'Verbo iterativo:':'Verbo iterativo:', 'Verbo defectivo:':'Verbo defectivo:',
         'Locucao':'Locução', 'Locugao':'Locução','Locuçáo prepositiva:':'Locução prepositiva:', 'Locucáo adverbial:':'Locução adverbial:',
         'Locucáo substantiva:':'Locução substantiva:', 'Locução substantiva:':'Locução substantiva:',
         'observagoes': 'observações', 'índío': 'índio', 'nao': 'não', 'Nao': 'Não', 'náo': 'não', 'Náo':'Não', 'hã':'há', 'homen':'homem',
         'fungáo': 'função', 'conjugagao': 'conjugação', 'Cabega': 'Cabeça', 'cabeqa': 'cabeça', 'cabe<ja':'cabeça', 'grao':'grão','señhorl':'senhor!',
         'composigao': 'composição', 'posipoes': 'posições', 'só ha': 'só há', ' ha ': ' há ', 'estã':'está','glande':'glande',
         'intradutlvel': 'intradutível', 'próxi mo': 'próximo', 'aqáo': 'ação', 'signíficado': 'significado','formagao':'formação',
         'á': 'ã', 'Á': 'Ã', '§': 'ç', '£': 'S', # Suposições para símbolos estranhos
         ' 9 ': ' ç ', '9 ': 'ç ', ' 9': ' ç', '(t, 9 )':'(t, ç)', # Correção mais agressiva para 9
         'ü': 'u', 'Ü': 'U', # Normalizar trema se desnecessário
         # 'y': 'y', # Manter Y com trema se for caractere válido no Tupi Antigo do dicionário
         ' l ': ' i ', # l isolado vira i? Cautela.
         'permiss^L': 'permissivo', '^fruto':'1 fruto', ' z ': ' 2 ', 'zGlande':'2 Glande','^contradizer':'1 contradizer', # Tenta numerar
         'si nal': 'sinal', 'planta (desenho)': 'planta (desenho)',
         ' prepoderante':' preponderante', ' oontrário':' contrário', ' riiinca':' nunca',
         ' afcsolutamente':' absolutamente', ' a/grupadas':' agrupadas', ' sicj':' sic ',
         'multidao': 'multidão', 'duplicagao': 'duplicação', 'duplicaqao': 'duplicação',
         'procedéncia': 'procedência', 'género': 'gênero', 'famllia': 'família', 'séxo': 'sexo',
         'mé' : 'mãe', 'Índío': 'índio', 'fago':'fago','recordagao':'recordação','padre':'Padre',
         'traduqáo': 'tradução', 'conjungáp': 'conjunção', 'Erigado':'Eriçado','Bispo':'Bispo',
         'diminutl:vo':'diminutivo','répleto':'repleto','pélos':'pelos','respiraqao':'respiração','memória':'memória',
         'fiaqao':'fiação','recepgáo':'recepção','AgÉ-AgEMA':'AGÉ-AGEMA', 'las.':'1as.', 'Botánica':'Botânica',
         'preposi^ gao':'preposição', 'pessbal':'pessoal','prepos_i gáo':'preposição', 'Adáo':'Adão',
         'abundáncia':'abundância','Preposdgáo':'Preposição','desen^Dlvem':'desenvolvem',
         'mao':'mão','cansago':'cansaço','conjugáo':'conjunção', 'oragáo':'oração', 'prSprio':'próprio',
         'oraqoes':'orações','ünica':'única','preposiqoes':'preposições','preposiqáo':'preposição',
         'entá o':'então','naqu_i':'naquilo','interroga tiva':'interrogativa', 'encru2ilhada':'encruzilhada',
         'fungoes':'funções','agao':'ação','posiqáo':'posição','explrlito':'espírito',
         'relaqao':'relação', 'engros- sar':'engrossar','solugáo':'solução',
         'ÍABA': 'ABA', 'f ': '† ', '+ ': '‡ ', # Usar símbolos distintos ou remover depois
         'tupi) . É': 'tupi). É', # Corrigir espaço antes de ponto final
         '(XE).': '(XE)', '(T-).': '(T-)', '(Q-).': '(Q-)', # Remover ponto após marcadores
         'homen':'homem', 'lingua':'língua', 'segundo':'segundo', # Correções gerais pt
         'homém':'homem', 'fácil':'fácil', 'ingreme':'íngreme', 'máo':'mão', # Mais pt
         'güloso':'guloso', 'trapaceiro':'trapaceiro', 'ladráo':'ladrão', 'descorogoa do':'descoroçoado',
         'córtez':'cortês', 'distinfto':'distinto', 'antropo fago':'antropófago','povo':'povo',
         'entranhas':'entranhas', 'pélo':'pelo','sobrinatural':'sobrenatural', 'tarróz':'arroz',
         'pjretos':'pretos', 'Particula':'Partícula', 'aemais':'demais',
         'slgnifica':'significa', 'sinónimo':'sinônimo', 'luslsmos':'lusismos', 'frema':'trema', 'Prefi xo':'Prefixo',
         'condicicnal':'condicional', 'adjetivos':'adjetivos','constitui-se':'constitui-se',
         'excrémenlo': 'excremento', 'portu- gués': 'português', 'porem': 'porém', 'pas- sado':'passado',
         'sé': 'se', 'ré': 'ré', 'é': 'é', 'í': 'i', 'Ó': 'ó', 'ó': 'ó', # Normalizar acentos se necessário
         'qüera':'güera', 'sübstantivo':'substantivo', 'egé':'eçé', # Exemplos de normalização ortográfica Tupi? (Verificar!)
         # ... (Adicionar CENTENAS de regras aqui!)
    }
    texto_processado = texto_processado.replace('(t, 9 )','(t, ç)') # Correção específica antes do loop
    for erro, correcao in correcoes.items():
        texto_processado = texto_processado.replace(erro, correcao) # Case-sensitive

    # Normalização final de espaços e pontuação
    texto_processado = re.sub(r'\s+', ' ', texto_processado).strip()
    texto_processado = re.sub(r'([.,;:?!])([a-zA-ZÀ-ú0-9])', r'\1 \2', texto_processado)
    texto_processado = re.sub(r'\s([.,;:?!])', r'\1', texto_processado)
    texto_processado = re.sub(r'\s+:\s+', ': ', texto_processado)
    texto_processado = re.sub(r'([a-zA-Z]):([a-zA-Z])', r'\1: \2', texto_processado)
    # Padronizar classes para terminar com ':' e espaço (se houver texto depois)
    for cl in CLASSES_GRAMATICAIS:
        # Substitui 'Classe.' por 'Classe: ' ou 'Classe ' por 'Classe: '
        texto_processado = re.sub(r'\b' + re.escape(cl) + r'\.(?!\S)', cl + ': ', texto_processado, flags=re.IGNORECASE)
        texto_processado = re.sub(r'\b' + re.escape(cl) + r'(?=\s|$)(?!:)', cl + ': ', texto_processado, flags=re.IGNORECASE)

    # Remover espaços extras após a padronização dos dois pontos
    texto_processado = re.sub(r':\s+', ': ', texto_processado)

    return texto_processado.strip()

print("Função de limpeza definida.")

# --- PASSO 5: Processamento e Extração Refinados (v6) ---
dados_estruturados = []
entradas_com_erro = []

# Lista de classes gramaticais (AJUSTE CONFORME SEU DICIONÁRIO LIMPO)
CLASSES_GRAMATICAIS = sorted([
    'Prefixo', 'Sufixo verbal', 'Sufixo', 'Afixo', 'Substantivo irregular', 'Substantivo', 'Gerúndio',
    'Verbo transitivo irregular', 'Verbo intransitivo irregular', 'Verbo transitivo relativo', 'Verbo transitivo',
    'Verbo intransitivo', 'Verbo predicativo', 'Verbo factivo', 'Verbo pronominal', 'Verbo reflexivo',
    'Verbo iterativo', 'Verbo defectivo', 'Verbo', 'Verbo durativo', 'Verbo freqüentativo', # Adicionando mais tipos
    'Vocativo', 'Demonstrativo', 'Partículas agrupadas', 'Partícula', 'Advérbio', 'Locução adverbial',
    'Adjetivo', 'Pronome interrogativo', 'Pronome pessoal', 'Pronome relativo', 'Pronome indefinido',
    'Pronome possessivo', 'Pronome objetivo', 'Pronome', # Genérico
    'Numeral', 'Interjeigáo', 'Conjungáo', 'Preposiqáo', 'Locuqáo prepositiva',
    'Locuqáo substantiva', 'Particípio', 'Ictiologia',
    'Ornitologia', 'Zoologia', 'Botánica', 'Entomologia', 'Conquiliologia',
    'Crustaceologia', 'Astronomia', 'Mineralogia', 'Mitologia', 'Malacologia',
    'Fitologia'
], key=len, reverse=True)
# Regex para encontrar uma dessas classes seguida por ':' (DEPOIS da limpeza robusta)
# Permite múltiplas palavras capitalizadas ou não antes dos dois pontos
regex_classe_precisa = re.compile(r'(\b(?:' + '|'.join(re.escape(c) for c in CLASSES_GRAMATICAIS) + r')(?:\s+[a-zA-Z]+)*):', re.IGNORECASE)

if 'entradas_brutas' in locals() and entradas_brutas:
    print(f"\n--- PASSO 5: Iniciando Processamento e Extração v6 de {len(entradas_brutas)} Entradas ---")
    for i, entrada_bruta in enumerate(entradas_brutas):
        if (i + 1) % 200 == 0: print(f"Processando entrada {i+1}/{len(entradas_brutas)}...")

        try:
            # Limpa a entrada inteira primeiro
            entrada_limpa = limpar_texto_robusto(entrada_bruta)
            if not entrada_limpa: continue

            verbete_tupi = ""
            marcadores = None
            classe_gramatical = None
            texto_para_definicoes = entrada_limpa # Começa com tudo
            primeira_linha_original = entrada_bruta.splitlines()[0].strip() if entrada_bruta else entrada_limpa

            # 1. Extrair Marcadores primeiro
            match_marcador = re.search(r'(\s*[({\[].*?[)}\]])', texto_para_definicoes)
            if match_marcador:
                marcadores = match_marcador.group(1).strip()
                start, end = match_marcador.span()
                texto_para_definicoes = (texto_para_definicoes[:start] + texto_para_definicoes[end:]).strip()

            # 2. Encontrar a PRIMEIRA Classe Gramatical + ':'
            match_classe_obj = regex_classe_precisa.search(texto_para_definicoes)
            pos_classe_inicio = match_classe_obj.start() if match_classe_obj else -1
            pos_classe_fim = match_classe_obj.end() if match_classe_obj else -1

            if pos_classe_inicio != -1:
                # Classe encontrada!
                classe_potencial = match_classe_obj.group(1).strip()
                # Validação (opcional mas seguro)
                if any(classe_potencial.lower() == c.lower() for c in CLASSES_GRAMATICAIS):
                    classe_gramatical = classe_potencial.capitalize()
                    # Verbete é tudo ANTES da classe
                    verbete_tupi = texto_para_definicoes[:pos_classe_inicio].strip()
                    # Texto para definições é tudo DEPOIS da classe (incluindo o ':')
                    texto_para_definicoes = texto_para_definicoes[pos_classe_fim:].strip()
                else: # Falso positivo
                    verbete_tupi = texto_para_definicoes # Assume tudo como verbete (será limpo)
                    texto_para_definicoes = ""
            else:
                # Nenhuma classe encontrada. Tenta separar pelo primeiro marcador de definição.
                # Procura por número/símbolo seguido de espaço OU início de EX/=.
                match_def_marker = re.search(r'\s(?:[1-9]+\.|†|‡|\^|z\b|EX:| = )', texto_para_definicoes)
                if match_def_marker:
                    verbete_tupi = texto_para_definicoes[:match_def_marker.start()].strip()
                    texto_para_definicoes = texto_para_definicoes[match_def_marker.start():].strip()
                else:
                    # Caso final: sem classe, sem marcador. Assume verbete é tudo.
                    verbete_tupi = texto_para_definicoes
                    texto_para_definicoes = ""


            # 3. Limpeza final do verbete (mais cuidadosa)
            # Remove apenas pontuação FINAL se não for precedida por letra/número (preserva A.)
            verbete_tupi = re.sub(r'(?<![a-zA-Z0-9])[:.—?]+\s*$', '', verbete_tupi).strip()
            # Remove traço inicial APENAS se seguido por espaço
            if verbete_tupi.startswith('— '): verbete_tupi = verbete_tupi[2:].strip()
            # Remove traço final APENAS se precedido por espaço
            if verbete_tupi.endswith(' —'): verbete_tupi = verbete_tupi[:-2].strip()


            # 4. Extrair Definições, Exemplos, Referências do texto_para_definicoes
            definicoes = []
            exemplos = []
            referencias = []

            regex_split_corpo = re.compile(r'(?=\s*(?:[1-9]+\.|†|‡|\^|z\b|EX:| = ))')
            partes_corpo = regex_split_corpo.split(texto_para_definicoes)

            for parte in partes_corpo:
                parte = parte.strip()
                if not parte: continue

                match_ex = re.match(r'^\s*EX:\s*(.*)', parte, re.IGNORECASE)
                match_ref = re.match(r'^\s*=\s*(.*)', parte)
                match_num = re.match(r'^\s*([1-9]+\.|†|‡|\^|z\b)\s*(.*)', parte) # Captura marcador e texto

                if match_ex:
                    texto_item = match_ex.group(1).strip()
                    if texto_item: exemplos.append(texto_item)
                elif match_ref:
                    texto_item = match_ref.group(1).strip()
                    if texto_item: referencias.append(texto_item)
                elif match_num:
                    texto_item = match_num.group(2).strip() # Pega só o texto após o marcador
                    if texto_item: definicoes.append(texto_item)
                else:
                    # Se não começou com marcador conhecido, assume que é definição
                     if parte: definicoes.append(parte)


            # 5. Armazenar dados estruturados
            if verbete_tupi or definicoes or exemplos or referencias:
                 if not re.fullmatch(r'[A-Z]\s*$', primeira_linha_original): # Filtra A, B, C..
                    dados_estruturados.append({
                        'id': len(dados_estruturados),
                        'verbete_original_linha': primeira_linha_original, # Usa original para referencia
                        'verbete_tupi': verbete_tupi,
                        'marcadores': marcadores,
                        'classe_gramatical': classe_gramatical,
                        'definicoes': definicoes,
                        'exemplos': exemplos,
                        'referencias': referencias,
                    })

        except Exception as e:
            print(f"\n!!!!!! Erro Crítico ao processar entrada bruta {i} !!!!!!")
            print(f"Erro: {e}")
            texto_erro = entrada_bruta[:1000] + "..." if len(entrada_bruta) > 1000 else entrada_bruta
            entradas_com_erro.append({"id_bruta": i, "erro": str(e), "texto": texto_erro})
            # raise e # Descomente para parar no primeiro erro grave

    print(f"\nProcessadas {len(dados_estruturados)} entradas estruturadas antes da filtragem final (v6).")
    print(f"Encontrados {len(entradas_com_erro)} erros durante o processamento.")

    # --- PASSO 6: Filtragem Final ---
    print("\n--- PASSO 6: Filtrando Entradas Vazias/Inválidas ---")
    dados_filtrados_final = [
        entry for entry in dados_estruturados
        if entry.get('verbete_tupi') and \
           (entry.get('classe_gramatical') or entry.get('definicoes') or entry.get('exemplos') or entry.get('referencias'))
    ]
    num_removidos_filtragem = len(dados_estruturados) - len(dados_filtrados_final)
    print(f"Removidas {num_removidos_filtragem} entradas durante a filtragem final.")
    dados_estruturados = dados_filtrados_final # Usa a lista filtrada

    # --- PASSO 7: Salvar Resultados ---
    print("\n--- PASSO 7: Salvando Resultados ---")
    # ----- Salvar os dados estruturados em JSON -----
    try:
        output_dir_json = os.path.dirname(caminho_arquivo_json)
        if output_dir_json and not os.path.exists(output_dir_json): print(f"Criando diretório de saída para JSON: {output_dir_json}"); os.makedirs(output_dir_json)
        with open(caminho_arquivo_json, 'w', encoding='utf-8') as f_json: json.dump(dados_estruturados, f_json, ensure_ascii=False, indent=2)
        print(f"\nDados estruturados finais ({len(dados_estruturados)} entradas) salvos em '{caminho_arquivo_json}'")
        if dados_estruturados:
            print("\n--- Exemplo das Primeiras 5 Entradas Estruturadas Finais (JSON v6) ---")
            for k in range(min(5, len(dados_estruturados))):
                 print(f"--- Entrada ID {dados_estruturados[k]['id']} ---"); print(json.dumps(dados_estruturados[k], ensure_ascii=False, indent=2)); print("-" * 20)
        else: print("\nNenhuma entrada estruturada válida para mostrar após filtragem.")
    except Exception as e: print(f"Erro ao salvar o arquivo JSON final: {e}")

    # ----- Salvar entradas com erro (se houver) -----
    if entradas_com_erro:
        try:
            output_dir_erro = os.path.dirname(caminho_arquivo_erros);
            if output_dir_erro and not os.path.exists(output_dir_erro): print(f"Criando diretório de saída para Erros: {output_dir_erro}"); os.makedirs(output_dir_erro)
            with open(caminho_arquivo_erros, 'w', encoding='utf-8') as f_err: json.dump(entradas_com_erro, f_err, ensure_ascii=False, indent=2)
            print(f"Informações de {len(entradas_com_erro)} erro(s) salvas em '{caminho_arquivo_erros}'")
        except Exception as e: print(f"Erro ao salvar arquivo de erros: {e}")

elif not 'entradas_brutas' in locals() or not entradas_brutas:
     print("Não há entradas brutas para processar. Verifique os Passos 1, 2 e 3.")

print("\n--- Processamento Concluído ---")

--- PASSO 1: Carregando Texto Bruto ---
Texto bruto carregado com sucesso de 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/arquivo_bruto_extraido.txt' usando encoding 'utf-8'.
Tamanho: 666298 caracteres.

--- PASSO 2: Iniciando Pré-filtragem em Memória ---
Pré-filtragem concluída. 10697 linhas removidas.
Tamanho do texto filtrado: 655031 caracteres.
Texto pré-filtrado (para debug) salvo em 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/arquivo_pre_filtrado_debug.txt'

--- PASSO 3: Dividindo Texto em Entradas ---
Encontrados 7946 potenciais inícios de entrada no texto filtrado.
Dividido em 7946 entradas brutas.

--- Exemplo das primeiras 5 entradas brutas (Após Divisão v7) ---
--- Entrada Bruta 1 ---
A
--------------------
--- Entrada Bruta 2 ---
AAN-GATU
--------------------
--- Entrada Bruta 3 ---
A
--------------------
--- Entrada Bruta 4 ---
A — . Prefi.xo: atua como pronome pessoal agente, da primei 
ra pessoa de verbos da segunda conjugagao (só ha 
duas 

In [20]:
import re
import json
import csv
import os

# --- Configurações ---
# Caminho para o JSON gerado pela v6
caminho_arquivo_json_v6 = 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/dicionario_estruturado_final_v6.json'

# Caminhos para os datasets de saída
caminho_dataset_txt_tupi = 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/dataset_tupi_v6.txt'
caminho_dataset_csv_paralelo = 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/dataset_paralelo_v6.csv'
caminho_output_dir = os.path.dirname(caminho_arquivo_json_v6) # Salva no mesmo dir do JSON

# Lista de classes (para tentar limpar do verbete) - Use a mesma da v6
CLASSES_GRAMATICAIS = sorted([
    'Prefixo', 'Sufixo verbal', 'Sufixo', 'Afixo', 'Substantivo irregular', 'Substantivo', 'Gerúndio',
    'Verbo transitivo irregular', 'Verbo intransitivo irregular', 'Verbo transitivo relativo', 'Verbo transitivo',
    'Verbo intransitivo', 'Verbo predicativo', 'Verbo factivo', 'Verbo pronominal', 'Verbo reflexivo',
    'Verbo iterativo', 'Verbo defectivo', 'Verbo', 'Verbo durativo', 'Verbo freqüentativo', 'Verbo birrelativo',
    'Vocativo', 'Demonstrativo', 'Partículas agrupadas', 'Partícula', 'Advérbio', 'Locução adverbial',
    'Adjetivo', 'Pronome interrogativo', 'Pronome pessoal', 'Pronome relativo', 'Pronome indefinido',
    'Pronome possessivo', 'Pronome objetivo', 'Pronome',
    'Numeral', 'Interjeigáo', 'Conjungáo', 'Preposiqáo', 'Locuqáo prepositiva',
    'Locuqáo substantiva', 'Particípio', 'Ictiologia',
    'Ornitologia', 'Zoologia', 'Botánica', 'Entomologia', 'Conquiliologia',
    'Crustaceologia', 'Astronomia', 'Mineralogia', 'Mitologia', 'Malacologia',
    'Fitologia', 'Anatomia'
], key=len, reverse=True)
# Regex para tentar encontrar o início de uma classe (para limpar o verbete)
regex_classe_inicio = re.compile(r'\s+(' + '|'.join(re.escape(c) for c in CLASSES_GRAMATICAIS) + r')[:.\s]', re.IGNORECASE)
# Regex para limpar marcadores de definição do início das strings
regex_limpa_def_inicio = re.compile(r'^\s*([1-9]+\.|†|‡|\^|z\b)\s*')

# --- Carregar o JSON v6 ---
print(f"--- Carregando JSON v6 de: {caminho_arquivo_json_v6} ---")
dados_estruturados = []
try:
    with open(caminho_arquivo_json_v6, 'r', encoding='utf-8') as f:
        dados_estruturados = json.load(f)
    print(f"JSON v6 carregado com {len(dados_estruturados)} entradas.")
except FileNotFoundError:
    print(f"Erro: Arquivo JSON v6 não encontrado em '{caminho_arquivo_json_v6}'")
    exit()
except json.JSONDecodeError as e:
    print(f"Erro ao decodificar o JSON v6: {e}")
    exit()
except Exception as e:
     print(f"Erro inesperado ao carregar JSON v6: {e}")
     exit()

# --- Gerar Dataset de Texto Puro (Tupi) ---
print(f"\n--- Gerando Dataset TXT (Tupi): {caminho_dataset_txt_tupi} ---")
count_linhas_txt = 0
try:
    with open(os.path.join(caminho_output_dir, caminho_dataset_txt_tupi), 'w', encoding='utf-8') as f_out:
        for entrada in dados_estruturados:
            verbete_bruto = entrada.get('verbete_tupi')
            if not verbete_bruto or not isinstance(verbete_bruto, str):
                continue

            # Tentar limpar o verbete removendo classe/definição que possa ter ficado
            verbete_limpo = verbete_bruto
            match_classe = regex_classe_inicio.search(verbete_limpo)
            if match_classe:
                verbete_limpo = verbete_limpo[:match_classe.start()]

            # Limpeza final de pontuação e traços (preservando hífens internos)
            verbete_limpo = re.sub(r'[:.—?]+\s*$', '', verbete_limpo).strip()
            if verbete_limpo.startswith('— '): verbete_limpo = verbete_limpo[2:].strip()
            if verbete_limpo.endswith(' —') and '-' not in verbete_limpo[:-2]: verbete_limpo = verbete_limpo[:-2].strip()

            verbete_limpo = verbete_limpo.strip()

            if verbete_limpo:
                f_out.write(verbete_limpo + '\n')
                count_linhas_txt += 1

    print(f"Dataset TXT (Tupi) gerado com {count_linhas_txt} linhas.")

except Exception as e:
    print(f"Erro ao gerar dataset TXT: {e}")


# --- Gerar Dataset Paralelo (CSV Tupi-Português) ---
print(f"\n--- Gerando Dataset CSV Paralelo: {caminho_dataset_csv_paralelo} ---")
count_linhas_csv = 0
try:
    with open(os.path.join(caminho_output_dir, caminho_dataset_csv_paralelo), 'w', newline='', encoding='utf-8') as f_csv:
        writer = csv.writer(f_csv)
        writer.writerow(['tupi', 'portugues']) # Cabeçalho

        for entrada in dados_estruturados:
            verbete_bruto = entrada.get('verbete_tupi')
            definicoes = entrada.get('definicoes', [])

            if not verbete_bruto or not isinstance(verbete_bruto, str) or not definicoes:
                continue

            # Limpar o verbete Tupi (mesma lógica do TXT)
            verbete_limpo = verbete_bruto
            match_classe = regex_classe_inicio.search(verbete_limpo)
            if match_classe:
                verbete_limpo = verbete_limpo[:match_classe.start()]
            verbete_limpo = re.sub(r'[:.—?]+\s*$', '', verbete_limpo).strip()
            if verbete_limpo.startswith('— '): verbete_limpo = verbete_limpo[2:].strip()
            if verbete_limpo.endswith(' —') and '-' not in verbete_limpo[:-2]: verbete_limpo = verbete_limpo[:-2].strip()
            verbete_limpo = verbete_limpo.strip()

            if not verbete_limpo: # Pula se o verbete ficou vazio após limpeza
                continue

            # Para cada definição, cria uma linha no CSV
            for definicao_bruta in definicoes:
                if not definicao_bruta or not isinstance(definicao_bruta, str):
                    continue

                # Limpar definição (remover classe gramatical do início e marcadores numéricos/símbolos)
                definicao_limpa = definicao_bruta
                # Tenta remover classe do início (case-insensitive)
                match_classe_def = regex_classe_precisa.match(definicao_limpa)
                if match_classe_def:
                    definicao_limpa = definicao_limpa[match_classe_def.end():].strip()

                # Remove marcadores numéricos/símbolos do início
                definicao_limpa = regex_limpa_def_inicio.sub('', definicao_limpa).strip()

                if definicao_limpa:
                    writer.writerow([verbete_limpo, definicao_limpa])
                    count_linhas_csv += 1

    print(f"Dataset CSV Paralelo gerado com {count_linhas_csv} pares.")

except Exception as e:
    print(f"Erro ao gerar dataset CSV: {e}")

print("\n--- Geração de Datasets Concluída (Baseada na v6) ---")

--- Carregando JSON v6 de: C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/dicionario_estruturado_final_v6.json ---
JSON v6 carregado com 6166 entradas.

--- Gerando Dataset TXT (Tupi): C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/dataset_tupi_v6.txt ---
Dataset TXT (Tupi) gerado com 6166 linhas.

--- Gerando Dataset CSV Paralelo: C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/dataset_paralelo_v6.csv ---
Dataset CSV Paralelo gerado com 6765 pares.

--- Geração de Datasets Concluída (Baseada na v6) ---


### Validação e Refinamento do JSON

In [ ]:
import json
import pandas as pd
import random

caminho_arquivo_json = 'dicionario_estruturado_final.json'
try:
    with open(caminho_arquivo_json, 'r', encoding='utf-8') as f:
        dados_estruturados = json.load(f)
    print(f"JSON carregado com {len(dados_estruturados)} entradas.")

    # Opcional: Converter para DataFrame Pandas para fácil visualização/filtragem
    df = pd.DataFrame(dados_estruturados)
    pd.set_option('display.max_rows', 100) # Mostrar mais linhas
    pd.set_option('display.max_colwidth', 200) # Mostrar mais texto nas colunas

    # Mostrar algumas entradas aleatórias
    print("\n--- Amostra Aleatória de Entradas (DataFrame Pandas) ---")
    print(df.sample(n=15)) # Veja 15 entradas aleatórias

    # Mostrar as primeiras e últimas entradas também pode ser útil
    # print("\n--- Primeiras 5 Entradas ---")
    # print(df.head(5))
    # print("\n--- Últimas 5 Entradas ---")
    # print(df.tail(5))

except FileNotFoundError:
    print(f"Erro: Arquivo JSON não encontrado em '{caminho_arquivo_json}'")
    dados_estruturados = None
    df = None
except json.JSONDecodeError as e:
    print(f"Erro ao decodificar o JSON: {e}")
    dados_estruturados = None
    df = None
except Exception as e:
     print(f"Erro inesperado ao carregar/processar JSON: {e}")
     dados_estruturados = None
     df = None

JSON carregado com 7922 entradas.

--- Amostra Aleatória de Entradas (DataFrame Pandas) ---
        id  \
4530  4530   
7671  7671   
6950  6950   
1933  1933   
5139  5139   
1991  1991   
7651  7651   
5879  5879   
5968  5968   
2688  2688   
1787  1787   
4811  4811   
3521  3521   
7125  7125   
4744  4744   

                                                                                     verbete_original_linha  \
4530                             MO-IO-BÃI-x0ARqA. Verbo transitivo: por um diante do outro. - mo-Io-bãi-a.   
7671                                                   YBYT-I-MBORA. Substantivo: pó, poeira. = ybytu-byra.   
6950                                                                  TAPERÃ. Ornitologia: andorinha grande   
1933                                                                EK0-PORANGA (T-). Substantivo: virtude,   
5139                       NDOaRA. Participio sufixo: o que costuma ser ou estar. S na- salacão de "guara".   
1991             

### Geração do Dataset

In [ ]:
import re
import json
import os
import pandas as pd
import random

# ==============================================================================
# --- Configurações ---
# ==============================================================================
print("--- Configurações ---")
caminho_arquivo_bruto = 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/arquivo_bruto_extraido.txt' # Use o seu caminho
caminho_arquivo_pre_filtrado_opcional = 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/arquivo_pre_filtrado_debug.txt' # Use o seu caminho
caminho_arquivo_json = 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/dicionario_estruturado_final_v8_cleanup.json' # <<<< Novo nome v8_cleanup
caminho_arquivo_erros = 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/entradas_com_erro_v8_cleanup.txt'         # <<<< Novo nome v8_cleanup

output_dir = os.path.dirname(caminho_arquivo_json)
if output_dir and not os.path.exists(output_dir):
    print(f"Criando diretório de saída: {output_dir}")
    os.makedirs(output_dir)

# ==============================================================================
# --- PASSO 1, 2, 3 (Carregar, Filtrar, Dividir - Mantido como antes) ---
# ==============================================================================
print("\n--- PASSO 1: Carregando Texto Bruto ---")
# ... (código de carregamento v7) ...
texto_completo_bruto = None
encodings_to_try = ['utf-8', 'latin-1', 'cp1252'];
for enc in encodings_to_try:
    try:
        with open(caminho_arquivo_bruto, 'r', encoding=enc) as f: texto_completo_bruto = f.read()
        print(f"Texto bruto carregado com sucesso de '{caminho_arquivo_bruto}' usando encoding '{enc}'.")
        print(f"Tamanho: {len(texto_completo_bruto)} caracteres."); break
    except FileNotFoundError: print(f"Erro Crítico: Arquivo bruto não encontrado em '{caminho_arquivo_bruto}'"); texto_completo_bruto = None; break
    except Exception as e: print(f"Falha ao ler com encoding '{enc}': {e}"); continue
if texto_completo_bruto is None: print("Erro Crítico: Não foi possível carregar o arquivo bruto."); exit()

print("\n--- PASSO 2: Iniciando Pré-filtragem em Memória ---")
# ... (código de pré-filtragem v7) ...
texto_filtrado_memoria = ""; linhas_removidas_count = 0
if texto_completo_bruto:
    linhas_brutas = texto_completo_bruto.splitlines(); linhas_filtradas = []; num_linhas_brutas = len(linhas_brutas)
    for i, linha in enumerate(linhas_brutas):
        linha_strip = linha.strip()
        if not linha_strip: linhas_removidas_count += 1; continue
        if re.fullmatch(r'\d+', linha_strip): linhas_removidas_count += 1; continue
        if re.fullmatch(r'[\*k \-]+', linha_strip) and len(linha_strip) > 3: linhas_removidas_count += 1; continue
        if i > 0 and i < num_linhas_brutas - 1:
             if re.fullmatch(r'[A-ZÁÉÍÓÚÇÑŸ\-]+\s*(?:/\s*[A-ZÁÉÍÓÚÇÑŸ\-]+\??|-)\s*', linha_strip, re.IGNORECASE):
                 linha_anterior_vazia = not linhas_brutas[i-1].strip(); linha_posterior_vazia = not linhas_brutas[i+1].strip()
                 if linha_anterior_vazia or linha_posterior_vazia or len(linhas_brutas[i-1]) < 15 or len(linhas_brutas[i+1]) < 15:
                    linhas_removidas_count += 1; continue
        if "Biblioteca Digital Curt Nimuendajú" in linha or "http://www.etnolinguistica.org" in linha:
            linhas_removidas_count += 1; continue
        linhas_filtradas.append(linha)
    texto_filtrado_memoria = "\n".join(linhas_filtradas)
    print(f"Pré-filtragem concluída. {linhas_removidas_count} linhas removidas."); print(f"Tamanho do texto filtrado: {len(texto_filtrado_memoria)} caracteres.")
    if caminho_arquivo_pre_filtrado_opcional:
        try:
            with open(caminho_arquivo_pre_filtrado_opcional, 'w', encoding='utf-8') as f_out: f_out.write(texto_filtrado_memoria)
            print(f"Texto pré-filtrado (para debug) salvo em '{caminho_arquivo_pre_filtrado_opcional}'")
        except Exception as e: print(f"Erro ao salvar arquivo pré-filtrado opcional: {e}")
else: print("Texto bruto não carregado.")

print("\n--- PASSO 3: Dividindo Texto em Entradas ---")
# ... (código de divisão v7) ...
entradas_brutas = []
if texto_filtrado_memoria:
    regex_inicio_entrada = re.compile(r"^(?:\s{0,2}(?:—\s)?(?:[A-ZÁÉÍÓÚÇÑŸ]|[ÁÉÍÓÚ]\b))|^(?:\s{0,2}[A-Z]\s*)$", re.MULTILINE)
    indices_inicio = [m.start() for m in regex_inicio_entrada.finditer(texto_filtrado_memoria)]
    if indices_inicio:
        print(f"Encontrados {len(indices_inicio)} potenciais inícios de entrada no texto filtrado.")
        for i in range(len(indices_inicio)):
            start = indices_inicio[i]; end = indices_inicio[i+1] if (i+1) < len(indices_inicio) else len(texto_filtrado_memoria)
            bloco = texto_filtrado_memoria[start:end].strip()
            if bloco: entradas_brutas.append(bloco)
        print(f"Dividido em {len(entradas_brutas)} entradas brutas.")
        # if entradas_brutas: # Comentado para reduzir output
        #     print("\n--- Exemplo das primeiras 5 entradas brutas (Após Divisão v8) ---")
        #     for i, entrada in enumerate(entradas_brutas[:5]): print(f"--- Entrada Bruta {i+1} ---"); print(entrada); print("-" * 20)
    else: print("Nenhum início de entrada encontrado com o padrão atual no texto filtrado.")
else: print("Texto filtrado em memória está vazio.")


# ==============================================================================
# --- PASSO 4: Função de Limpeza Robusta (v8 - EXPANDIR!) ---
# ==============================================================================
print("\n--- PASSO 4: Definindo Função de Limpeza v8 ---")
# Lista de classes gramaticais (precisa ser definida globalmente)
CLASSES_GRAMATICAIS_LIST = sorted([
    'Prefixo', 'Sufixo verbal', 'Sufixo', 'Afixo', 'Substantivo irregular', 'Substantivo', 'Gerúndio',
    'Verbo transitivo irregular', 'Verbo intransitivo irregular', 'Verbo transitivo relativo', 'Verbo transitivo',
    'Verbo intransitivo', 'Verbo predicativo', 'Verbo factivo', 'Verbo pronominal', 'Verbo reflexivo',
    'Verbo iterativo', 'Verbo defectivo', 'Verbo', 'Verbo durativo', 'Verbo freqüentativo', 'Verbo birrelativo',
    'Vocativo', 'Demonstrativo', 'Partículas agrupadas', 'Partícula', 'Advérbio', 'Locução adverbial',
    'Adjetivo', 'Pronome interrogativo', 'Pronome pessoal agente', 'Pronome pessoal', 'Pronome relativo', 'Pronome indefinido',
    'Pronome possessivo', 'Pronome objetivo', 'Pronome',
    'Numeral cardinal', 'Numeral ordinal', 'Numeral distributivo', 'Numeral',
    'Interjeigáo', 'Interjeição', 'Conjungáo', 'Conjunção', 'Preposiqáo', 'Preposição', 'Locuqáo prepositiva',
    'Locuqáo adverbial', 'Locuqáo substantiva', 'Particípio', 'Ictiologia',
    'Ornitologia', 'Zoologia', 'Botánica', 'Entomologia', 'Conquiliologia',
    'Crustaceologia', 'Astronomia', 'Mineralogia', 'Mitologia', 'Malacologia',
    'Fitologia', 'Anatomia'
], key=len, reverse=True)

def limpar_texto_robusto(texto, classes_para_padronizar):
    if not texto: return ""
    texto_processado = " ".join(line.strip() for line in texto.splitlines() if line.strip())

    # !!!!!!!!!! EXPANDA MASSIVAMENTE ESTE DICIONÁRIO !!!!!!!!!!
    correcoes = {
        # Erros comuns de OCR e Acentuação (exemplos)
         'Prefi. xo': 'Prefixo', 'fiubstantivo': 'Substantivo', 'Substarítivo': 'Substantivo', 'S)ibsta\'ntivo': 'Substantivo',
         'Substantlvo': 'Substantivo', 'Sübstantivo': 'Substantivo', 'Substantivo irregular:':'Substantivo irregular:', 'fíubstantivo':'Substantivo', 'Substantivo:':'Substantivo:',
         'Adj^fcivo': 'Adjetivo', 'A_d jetivo':'Adjetivo', 'Mjetivao': 'Adjetivo','Adjetivo:':'Adjetivo:',
         'Partlculas': 'Partículas', 'Patticulas': 'Partículas', 'Partícúla': 'Partícula', 'Particlpio':'Particípio', 'Particípio':'Particípio:', 'Particulas':'Partículas','Particulas agrupadas':'Partículas agrupadas:',
         'Advérbiorse': 'Advérbio: se','Adverbio:': 'Advérbio:', 'Adverbio': 'Advérbio', 'Advérbio:':'Advérbio:',
         'Verbotransitivo': 'Verbo transitivo', 'Verbo intransitivo:o': 'Verbo intransitivo:', 'Verbo intransitivo :':'Verbo intransitivo:',
         'Verbo paredioativo': 'Verbo predicativo', 'Verbo transitivo:':'Verbo transitivo:', 'Verbo intransitivo:':'Verbo intransitivo:',
         'Verbo reflexivo:':'Verbo reflexivo:', 'Verbo predicativo:':'Verbo predicativo:', 'Verbo factivo:':'Verbo factivo:',
         'Verbo pronominal:':'Verbo pronominal:', 'Verbo iterativo:':'Verbo iterativo:', 'Verbo defectivo:':'Verbo defectivo:', 'Verbo:':'Verbo:',
         'Locucao':'Locução', 'Locugao':'Locução','Locuçáo prepositiva:':'Locução prepositiva:', 'Locucáo adverbial:':'Locução adverbial:',
         'Locucáo substantiva:':'Locução substantiva:', 'Locução substantiva:':'Locução substantiva:', 'Locução prepositiva:':'Locução prepositiva:',
         'Locução adverbial:':'Locução adverbial:',
         'Pronome interrogativo:': 'Pronome interrogativo:', 'Pronome pessoal:': 'Pronome pessoal:', 'Pronome pessoal agente:':'Pronome pessoal agente:',
         'Pronome relativo:': 'Pronome relativo:', 'Pronome indefinido:': 'Pronome indefinido:',
         'Pronome possessivo:': 'Pronome possessivo:', 'Pronome objetivo:': 'Pronome objetivo:',
         'Pronome:': 'Pronome:',
         'Numeral:': 'Numeral:', 'Interjeigáo:': 'Interjeição:', 'Conjungáo:': 'Conjunção:', 'Conjungao:':'Conjunção:',
         'Preposiqáo:': 'Preposição:', 'Preposicáo:': 'Preposição:', 'Ictiologia:':'Ictiologia:',
         'Ornitologia:':'Ornitologia:', 'Zoologia:':'Zoologia:', 'Botánica:':'Botânica:',
         'Entomologia:':'Entomologia:', 'Conquiliologia:':'Conquiliologia:',
         'Crustaceologia:':'Crustaceologia:', 'Astronomia:':'Astronomia:', 'Mineralogia:':'Mineralogia:',
         'Mitologia:':'Mitologia:', 'Malacologia:':'Malacologia:', 'Fitologia:':'Fitologia:', 'Anatomia:':'Anatomia:',
         'Vocativo:':'Vocativo:', 'Gerúndio:':'Gerúndio:', 'Demonstrativo:':'Demonstrativo:',
         'Afixo:':'Afixo:', 'Sufixo:':'Sufixo:', 'Sufixo verbal:':'Sufixo verbal:', 'Numeral cardinal:':'Numeral cardinal:',
         'Numeral ordinal:':'Numeral ordinal:', 'Numeral distributivo:':'Numeral distributivo:',
         # --- Erros de OCR e Acentuação ---
         'observagoes': 'observações', 'índío': 'índio', 'nao': 'não', 'Nao': 'Não', 'náo': 'não', 'Náo':'Não', 'hã':'há', 'homen':'homem',
         'fungáo': 'função', 'conjugagao': 'conjugação', 'Cabega': 'Cabeça', 'cabeqa': 'cabeça', 'cabe<ja': 'cabeça', 'grao':'grão','señhorl':'senhor!',
         'composigao': 'composição', 'posipoes': 'posições', 'só ha': 'só há', ' ha ': ' há ', 'estã':'está','glande':'glande', 'ami- go':'amigo',
         'intradutlvel': 'intradutível', 'próxi mo': 'próximo', 'aqáo': 'ação', 'signíficado': 'significado','formagao':'formação',
         'á': 'ã', 'Á': 'Ã', '§': 'ç', '£': 'S',
         ' 9 ': ' ç ', '9 ': 'ç ', ' 9': ' ç', '(t, 9 )':'(t, ç)',
         'ü': 'u', 'Ü': 'U',
         ' l ': ' i ',
         'permiss^L': 'permissivo', '^fruto':'1 fruto', ' z ': ' 2 ', 'zGlande':'2 Glande','^contradizer':'1 contradizer',
         'si nal': 'sinal', 'planta (desenho)': 'planta (desenho)', 'prepoderante': 'preponderante', 'oontrário': 'contrário', 'riiinca': 'nunca',
         'afcsolutamente': 'absolutamente', 'a/grupadas': 'agrupadas', 'sicj': 'sic', 'multidao': 'multidão',
         'duplicagao': 'duplicação', 'duplicaqao': 'duplicação', 'procedéncia': 'procedência', 'género': 'gênero',
         'famllia': 'família', 'séxo': 'sexo', 'mé': 'mãe', 'fago': 'fago', 'recordagao': 'recordação', 'padre': 'Padre',
         'traduqáo': 'tradução', 'conjungáp': 'conjunção', 'Erigado': 'Eriçado', 'Bispo': 'Bispo', 'tupi) . É':'tupi). É',
         'diminutl:vo': 'diminutivo', 'répleto': 'repleto', 'pélos': 'pelos', 'respiraqao': 'respiração', 'memória': 'memória',
         'fiaqao': 'fiação', 'recepgáo': 'recepção', 'AgÉ-AgEMA': 'AGÉ-AGEMA', 'las.': '1as.',
         'preposi^ gao': 'preposição', 'pessbal': 'pessoal', 'prepos_i gáo': 'preposição', 'Adáo': 'Adão',
         'abundáncia': 'abundância', 'Preposdgáo': 'Preposição', 'desen^Dlvem': 'desenvolvem',
         'cansago': 'cansaço', 'conjugáo': 'conjunção', 'oragáo': 'oração', 'prSprio': 'próprio', 'oragoes': 'orações',
         'ünica': 'única', 'preposiqoes': 'preposições', 'preposiqáo': 'preposição', 'entá o': 'então',
         'naqu_i': 'naquilo', 'interroga tiva': 'interrogativa', 'encru2ilhada': 'encruzilhada', 'fungoes': 'funções',
         'agao': 'ação', 'posiqáo': 'posição', 'explrlito': 'espírito', 'relaqao': 'relação', 'engros- sar': 'engrossar',
         'solugáo': 'solução', 'ÍABA': 'ABA', 'f ': '† ', '+ ': '‡ ',
         '(XE).': '(XE)', '(T-).': '(T-)', '(Q-).': '(Q-)', '(C-).': '(C-)', '(IO-).': '(IO-)', '(NH-).': '(NH-)', '(I-).': '(I-)',
         'lingua': 'língua', 'segundo': 'segundo', 'homém': 'homem', 'fácil': 'fácil', 'ingreme': 'íngreme', 'máo': 'mão',
         'güloso': 'guloso', 'trapaceiro': 'trapaceiro', 'ladráo': 'ladrão', 'descorogoa do': 'descoroçoado',
         'córtez': 'cortês', 'distinfto': 'distinto', 'antropo fago': 'antropófago', 'povo': 'povo',
         'entranhas': 'entranhas', 'pélo': 'pelo', 'sobrinatural': 'sobrenatural', 'tarróz': 'arroz',
         'pjretos': 'pretos', 'Particula': 'Partícula', 'aemais': 'demais', 'slgnifica': 'significa',
         'sinónimo': 'sinônimo', 'luslsmos': 'lusismos', 'frema': 'trema', 'condicicnal': 'condicional',
         'adjetivos': 'adjetivos', 'constitui-se': 'constitui-se', 'excrémenlo': 'excremento', 'portu- gués': 'português',
         'porem': 'porém', 'pas- sado': 'passado', 'tupi).É': 'tupi). É', 'em tupi).É': 'em tupi). É', # Mais correções de espaço
        # ... (Adicione mais!)
    }

    # Aplica correções
    texto_processado = texto_processado.replace('(t, 9 )', '(t, ç)')
    for erro, correcao in correcoes.items():
        texto_processado = texto_processado.replace(erro, correcao)

    # Normalização final
    texto_processado = re.sub(r'\s+', ' ', texto_processado).strip()
    texto_processado = re.sub(r'([.,;:?!])([a-zA-Z0-9])', r'\1 \2', texto_processado)
    texto_processado = re.sub(r'\s([.,;:?!])', r'\1', texto_processado)
    texto_processado = re.sub(r'\s+:\s+', ': ', texto_processado)
    texto_processado = re.sub(r'([a-zA-Z]):([a-zA-Z])', r'\1: \2', texto_processado)

    # Padronizar classes conhecidas para terminar com ':'
    for cl in classes_para_padronizar:
        pattern_end_dot = r'\b' + re.escape(cl) + r'\.(?=\s|$)'
        pattern_end_space = r'\b' + re.escape(cl) + r'(?=\s|$)(?!:)'
        texto_processado = re.sub(pattern_end_dot, cl + ':', texto_processado, flags=re.IGNORECASE)
        texto_processado = re.sub(pattern_end_space, cl + ':', texto_processado, flags=re.IGNORECASE)
    texto_processado = re.sub(r':\s+', ': ', texto_processado)

    return texto_processado.strip()

print("Função de limpeza definida.")

# --- PASSO 5: Processamento e Extração Refinados (v8) ---
dados_estruturados = []
entradas_com_erro = []

# Regex aprimorada para classe: permite espaços e hífens, EXIGE ':' no final
regex_classe_precisa_v8 = re.compile(r'(\b(?:' + '|'.join(re.escape(c) for c in CLASSES_GRAMATICAIS_LIST) + r')(?:\s+[-\w]+)*):', re.IGNORECASE)
# Regex para limpar marcadores do início das definições
regex_limpa_def_inicio_v8 = re.compile(r'^\s*(?:[1-9]+\.|†|‡|\^|z\b)\s*')

if 'entradas_brutas' in locals() and entradas_brutas:
    print(f"\n--- PASSO 5: Iniciando Processamento e Extração v8 de {len(entradas_brutas)} Entradas ---")
    for i, entrada_bruta in enumerate(entradas_brutas):
        if (i + 1) % 200 == 0: print(f"Processando entrada {i+1}/{len(entradas_brutas)}...")

        try:
            # Limpa a entrada inteira PRIMEIRO
            entrada_limpa = limpar_texto_robusto(entrada_bruta, CLASSES_GRAMATICAIS_LIST)
            if not entrada_limpa: continue

            verbete_tupi = ""
            marcadores = None
            classe_gramatical = None
            texto_para_definicoes = entrada_limpa
            primeira_linha_original = entrada_bruta.splitlines()[0].strip() if entrada_bruta else entrada_limpa

            # 1. Extrair Marcadores (...) primeiro
            match_marcador = re.search(r'(\s*[({\[].*?[)}\]])', texto_para_definicoes)
            if match_marcador:
                marcadores = match_marcador.group(1).strip()
                start, end = match_marcador.span()
                texto_para_definicoes = (texto_para_definicoes[:start] + texto_para_definicoes[end:]).strip()

            # 2. Encontrar a PRIMEIRA Classe Gramatical + ':' no texto restante
            match_classe_obj = regex_classe_precisa_v8.search(texto_para_definicoes)
            pos_classe_inicio = match_classe_obj.start() if match_classe_obj else -1

            if pos_classe_inicio != -1:
                # Classe encontrada!
                classe_achada = match_classe_obj.group(1).strip()
                classe_gramatical = classe_achada.capitalize()
                # Verbete é tudo ANTES da classe
                verbete_tupi = texto_para_definicoes[:pos_classe_inicio].strip()
                # Texto para definições é tudo DEPOIS do match COMPLETO (incluindo o ':')
                texto_para_definicoes = texto_para_definicoes[match_classe_obj.end():].strip()
            else:
                # Nenhuma classe encontrada. Tenta separar por marcador de definição/EX/=.
                match_def_marker = re.search(r'\s(?:[1-9]+\.|†|‡|\^|z\b|EX:| = )', texto_para_definicoes)
                if match_def_marker:
                    verbete_tupi = texto_para_definicoes[:match_def_marker.start()].strip()
                    texto_para_definicoes = texto_para_definicoes[match_def_marker.start():].strip()
                else:
                    # Assume verbete é tudo.
                    verbete_tupi = texto_para_definicoes
                    texto_para_definicoes = ""


            # 3. Limpeza final do verbete
            # Remove APENAS ':' ou '?' do final da string
            verbete_tupi = re.sub(r'[:?]\s*$', '', verbete_tupi).strip()
            # Remove traço inicial APENAS se seguido por espaço OU se for o único caractere não-espaço
            if verbete_tupi.startswith('— ') or verbete_tupi == '—': verbete_tupi = verbete_tupi[1:].strip()
            # Remove traço final APENAS se precedido por espaço E não houver hífen interno
            if verbete_tupi.endswith(' —') and '-' not in verbete_tupi[:-2]: verbete_tupi = verbete_tupi[:-2].strip()
            # Remove ponto final se não for precedido por letra maiúscula (para preservar abreviações como "A.")
            if verbete_tupi.endswith('.') and (len(verbete_tupi) <= 2 or not verbete_tupi[-2].isupper()):
                verbete_tupi = verbete_tupi[:-1].strip()


            # 4. Extrair Definições, Exemplos, Referências
            definicoes = []
            exemplos = []
            referencias = []

            regex_split_corpo = re.compile(r'(?=\s*(?:[1-9]+\.|†|‡|\^|z\b|EX:| = ))')
            partes_corpo = regex_split_corpo.split(texto_para_definicoes)

            for parte in partes_corpo:
                parte = parte.strip()
                if not parte: continue

                match_ex = re.match(r'^\s*EX:\s*(.*)', parte, re.IGNORECASE)
                match_ref = re.match(r'^\s*=\s*(.*)', parte)
                match_num = re.match(r'^\s*(?:[1-9]+\.|†|‡|\^|z\b)\s*(.*)', parte) # Captura SÓ o texto

                if match_ex:
                    texto_item = match_ex.group(1).strip();
                    if texto_item: exemplos.append(texto_item)
                elif match_ref:
                    texto_item = match_ref.group(1).strip();
                    if texto_item: referencias.append(texto_item)
                elif match_num:
                    texto_item = match_num.group(1).strip(); # Pega o texto após o marcador
                    if texto_item: definicoes.append(texto_item)
                else:
                     # Se não começou com marcador, é definição
                     if parte: definicoes.append(parte)


            # 5. Armazenar dados estruturados
            if verbete_tupi or definicoes or exemplos or referencias:
                 if not re.fullmatch(r'[A-Z]\s*$', primeira_linha_original): # Filtra A, B, C..
                    dados_estruturados.append({
                        'id': len(dados_estruturados),
                        'verbete_original_linha': primeira_linha_original,
                        'verbete_tupi': verbete_tupi,
                        'marcadores': marcadores,
                        'classe_gramatical': classe_gramatical,
                        'definicoes': definicoes,
                        'exemplos': exemplos,
                        'referencias': referencias,
                    })

        except Exception as e:
            print(f"\n!!!!!! Erro Crítico ao processar entrada bruta {i} !!!!!!")
            print(f"Erro: {e}")
            texto_erro = entrada_bruta[:1000] + "..." if len(entrada_bruta) > 1000 else entrada_bruta
            entradas_com_erro.append({"id_bruta": i, "erro": str(e), "texto": texto_erro})

    print(f"\nProcessadas {len(dados_estruturados)} entradas estruturadas antes da filtragem final (v8).")
    print(f"Encontrados {len(entradas_com_erro)} erros durante o processamento.")

    # --- PASSO 6: Filtragem Final ---
    print("\n--- PASSO 6: Filtrando Entradas Vazias/Inválidas ---")
    dados_filtrados_final = [
        entry for entry in dados_estruturados
        if entry.get('verbete_tupi') and \
           (entry.get('classe_gramatical') or entry.get('definicoes') or entry.get('exemplos') or entry.get('referencias'))
    ]
    num_removidos_filtragem = len(dados_estruturados) - len(dados_filtrados_final)
    print(f"Removidas {num_removidos_filtragem} entradas durante a filtragem final.")
    dados_estruturados = dados_filtrados_final # Usa a lista filtrada

    # --- PASSO 7: Salvar Resultados ---
    # (Código de salvamento JSON e Erros mantido como antes)
    print("\n--- PASSO 7: Salvando Resultados ---")
    try:
        if output_dir and not os.path.exists(output_dir): print(f"Criando diretório de saída para JSON: {output_dir}"); os.makedirs(output_dir)
        with open(caminho_arquivo_json, 'w', encoding='utf-8') as f_json: json.dump(dados_estruturados, f_json, ensure_ascii=False, indent=2)
        print(f"\nDados estruturados finais ({len(dados_estruturados)} entradas) salvos em '{caminho_arquivo_json}'")
        if dados_estruturados:
            print("\n--- Exemplo das Primeiras 10 Entradas Estruturadas Finais (JSON v8) ---")
            for k in range(min(10, len(dados_estruturados))):
                 print(f"--- Entrada ID {dados_estruturados[k]['id']} ---"); print(json.dumps(dados_estruturados[k], ensure_ascii=False, indent=2)); print("-" * 20)
        else: print("\nNenhuma entrada estruturada válida para mostrar após filtragem.")
    except Exception as e: print(f"Erro ao salvar o arquivo JSON final: {e}")

    if entradas_com_erro:
        try:
            output_dir_erro = os.path.dirname(caminho_arquivo_erros);
            if output_dir_erro and not os.path.exists(output_dir_erro): print(f"Criando diretório de saída para Erros: {output_dir_erro}"); os.makedirs(output_dir_erro)
            with open(caminho_arquivo_erros, 'w', encoding='utf-8') as f_err: json.dump(entradas_com_erro, f_err, ensure_ascii=False, indent=2)
            print(f"Informações de {len(entradas_com_erro)} erro(s) salvas em '{caminho_arquivo_erros}'")
        except Exception as e: print(f"Erro ao salvar arquivo de erros: {e}")

elif not 'entradas_brutas' in locals() or not entradas_brutas:
     print("Não há entradas brutas para processar. Verifique os Passos 1, 2 e 3.")

print("\n--- Processamento Concluído ---")


# --- PASSO 8: Validação Rápida com Pandas (Opcional) ---
# (Código de validação mantido como antes)
print("\n--- PASSO 8: Iniciando Validação Rápida com Pandas (Opcional) ---")
if os.path.exists(caminho_arquivo_json):
    try:
        df_final = pd.read_json(caminho_arquivo_json)
        print(f"JSON final carregado no Pandas com {len(df_final)} entradas.")
        pd.set_option('display.max_rows', 100); pd.set_option('display.max_colwidth', 200)
        print("\n--- Amostra Aleatória Final (15 entradas) ---")
        num_amostras = min(15, len(df_final))
        if num_amostras > 0: print(df_final.sample(n=num_amostras))
        else: print("DataFrame final vazio.")
        print("\n--- Contagem de Nulos por Coluna ---"); print(df_final.isnull().sum())
        if 'classe_gramatical' in df_final.columns: print("\n--- Exemplo de Entradas SEM Classe Gramatical ---"); print(df_final[df_final['classe_gramatical'].isnull()].head())
        if 'definicoes' in df_final.columns: print("\n--- Exemplo de Entradas COM Definições Vazias ---"); print(df_final[df_final['definicoes'].apply(lambda x: not x)].head()) # Verifica lista vazia
    except Exception as e: print(f"Erro durante a validação com Pandas: {e}")
else: print(f"Arquivo JSON final '{caminho_arquivo_json}' não encontrado para validação.")

--- Configurações ---

--- PASSO 1: Carregando Texto Bruto ---
Texto bruto carregado com sucesso de 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/arquivo_bruto_extraido.txt' usando encoding 'utf-8'.
Tamanho: 666298 caracteres.

--- PASSO 2: Iniciando Pré-filtragem em Memória ---
Pré-filtragem concluída. 10697 linhas removidas.
Tamanho do texto filtrado: 655031 caracteres.
Texto pré-filtrado (para debug) salvo em 'C:/Users/peterson.pereira/Documents/GitHub/Tupi2LLM/output/arquivo_pre_filtrado_debug.txt'

--- PASSO 3: Dividindo Texto em Entradas ---
Encontrados 7946 potenciais inícios de entrada no texto filtrado.
Dividido em 7946 entradas brutas.

--- Exemplo das primeiras 5 entradas brutas (Após Divisão v9) ---
--- Entrada Bruta 1 ---
A
--------------------
--- Entrada Bruta 2 ---
AAN-GATU
--------------------
--- Entrada Bruta 3 ---
A
--------------------
--- Entrada Bruta 4 ---
A — . Prefi.xo: atua como pronome pessoal agente, da primei 
ra pessoa de verbos da segunda c